# **Tarea 4 - CC6205 Natural Language Processing 📚**

Bienvenid@s a la cuarta tarea del curso de Natural Language Processing (NLP).
En esta tarea estaremos tratando el problema de **tagging** (generación de secuencias de etiquetas del mismo largo que la secuencia de input), el uso de **Convolutional Neural Networks** y **Recurrent Neural Networks**, e implementaremos una red usando PyTorch.

# Hidden Markov Models (HMM), Maximum Entropy Markov Models (MEMM) and Conditional Random Field(CRF) (1,5 puntos)

### Pregunta 1 (1 pt)
Para un problema de POS tagging se define el conjunto de etiquetas $S = \{ \text{DET}, \text{NOUN}, \text{VERB}, \text{ADP} \}$ y se tiene un Hidden Markov Model con los siguientes parámetros estimados a partir de un corpus de entrenamiento:

\begin{equation}
\begin{split}
q(\text{NOUN}| \text{ VERB}, \text{DET}) &= 0.3 \\
q(\text{NOUN}|\ w, \text{DET}) &= 0 \qquad \forall w \in S, w \neq \text{VERB} \\
q(\text{DET}| \text{ VERB}, \text{NOUN}) &= 0.4 \\
q(\text{DET}|\ w, \text{NOUN}) &= 0 \qquad \forall w \in S, w \neq \text{VERB} \\
e(the|\text{ DET}) &= 0.5 \\
e(pasta|\text{ NOUN}) &= 0.6
\end{split}
\end{equation}

Luego para la oración: `the man is pouring sauce on the pasta`, se tiene una tabla de programación dinámica con los siguientes valores:
\begin{equation}
\begin{split}
\pi(7,\text{DET},\text{DET})&=0.1\\
\pi(7,\text{NOUN},\text{DET})&=0.2\\
\pi(7,\text{VERB},\text{DET})&=0.01\\
\pi(7,\text{ADP},\text{DET})&=0.5
\end{split}
\end{equation}

Con esta información, calcule el valor de $\pi(8,\text{DET},\text{NOUN})$. Puede dejar el resultado expresado como una fracción.


>En base a la definición recursiva, se busca calcular lo siguiente:
>
>\begin{equation}
>\begin{split}
>\pi(8,\text{DET},\text{NOUN}) & = max_{w\in S_{6}} ( \pi(7,w,\text{DET}) \times q(\text{NOUN}|w,\text{DET}) \times q(\text{pasta}|\text{NOUN}) )\\
>\end{split}
>\end{equation}
>
>
>Se tiene directamente el valor de $q(\text{pasta}|\text{NOUN})=0.6$.
>Luego, es importante considerar que $q(\text{NOUN}|w,\text{DET}) = 0$ para los casos $w \neq \text{VERB}$ y que $\pi(7,w,\text{DET}) > 0$ para $w \in S$.
>
>Combinando ambos casos se llega a que $\pi(7,w,\text{DET}) \times q(\text{NOUN}|w,\text{DET}) > 0$ solamente para $w=\text{VERB}$.
>Por ende, se concluye que el argumento se maximiza con $w=\text{VERB}$.
>
>Luego, la ecuación queda como:
>
>\begin{equation}
>\begin{split}
>\pi(8,\text{DET},\text{NOUN}) & = \pi(7,\text{VERB},\text{DET}) \times q(\text{NOUN}|\text{VERB},\text{DET}) \times 0.6 \\
>& = \pi(7,\text{VERB},\text{DET}) \times 0.3 \times 0.6\\
>& = 0.01 \times 0.3 \times 0.6\\
>\pi(8,\text{DET},\text{NOUN}) & = 0.0018
>\end{split}
>\end{equation}

### Pregunta 2 (0.5 pts)
Comente  sobre las similitudes o diferencias entre los HMMs, MEMMs y CRFs. Para esto, responda las siguientes preguntas.

#### 2.1. ¿Para qué tipo de tarea sirven? Dé dos ejemplo de este tipo de tarea y descríbalos brevemente. (0.1 pts)

>Las tareas que pueden resolver estos modelos involucran etiquetado de secuencias, como por ejemplo POS tagging (*part of speech tagging*) y NER (*Named Entity Recognition*). POS Tagging entrega un tag a cada palabra de un texto, indicando el tipo de palabra (adverbio, sustantivo, verbo, etc). Por otra parte, NER identifica los pronombres personales de un texto y los asocia con lugares, personas, empresas, etc. Un ejemplo de ello es "[...] *New York* [...]", con etiquetas *Start Location* y *Continue Location*, respectivamente.

#### 2.2. ¿Qué modelos usan features? ¿Qué ventajas conlleva esto? (0.1 pts)

>Los modelos que usan features son MEMMs y CRFs. Esto tiene la ventaja de que pueden capturar relaciones de la entrada con las etiquetas correspondientes, lo que puede aportar información muy valiosa para determinar la secuencia de etiquetas. 

#### 2.3. ¿Cómo maneja cada uno de los modelos las palabras con baja frecuencia en el set de train? (0.1 pts)

>HMM divide el conjunto en palabras en baja y alta frecuencia. Las palabras de baja frecuencia se agrupan en distintos subconjuntos, de modo que su probabilidad sea mayor en comparación al conteo de la palabra sin agrupar. Un ejemplo de ello son los números de 2 dígitos, que se agrupan todos como si fuesen un mismo token.
>
>Como MEMM y CRF son modelos discriminativos, no tratan de forma directa el problema de palabras con baja frecuencia debido a que no necesitan estimar la probabilida de las palabras, sino que estiman una probabilidad condicional por medio de vectores de características.

#### 2.4. ¿Qué le permite a los CRF realizar decisiones globales? ¿Qué diferencia con respecto a los MEMMs permite lograr esto? ¿Por qué los HMMs tampoco son capaces de tomar decisiones globales? (0.1 pts)

>Los CRF pueden tomar decisiones globales debido a la normalización que presentan. Dicha normalización es global, ya que considera todas las combinaciones de secuencias posibles, capturando mucha más información. En el caso de MEMMs, su normalización es local, por lo que logra mirar solo un par de etiquetas atrás. Esto produce que MEMMs pierda mucho del contexto por su falta de memoria, pero que CRFs pueda capturarlo de mejor manera.
>
>En el caso de los HMMs tampoco tiene memoria global. Esto se debe a que a diferencia de CRF y MEMMs, no se relaciona el input con las etiquetas, lo cual no considera el contexto de la secuencia de entrada pues no tiene una función $\phi$.

#### 2.5 Dado una secuencia de $x_1, ..., x_m$ ¿Cuántas posibles secuencias de etiquetas se pueden generar para un conjunto de etiquetas $S$ con $|S|=k$ ? ¿Analizarlas todas sería computacionalmente tratable? (0.1 pts)

>Un secuencia de tamaño $m$ con $k$ opciones distintas para cada elemento de la secuencia, posee un total de $k^m$ posibles secuencias. Esto quiere decir que el número posible de secuencias crece exponencialmente a medida que aumenta el número de tokens. Es por lo anterior que se vuelve intratable intentar estimar por fuerza bruta cada una de las secuencias para la mayoría de los problemas, motivo por el que se usa un modelo generativo como HMM, o modelos discriminativos como MEMMs o CRFs.

# Convolutional Neural Networks (0,5 puntos)

### Pregunta 3 (0,5 puntos)

Considere la frase $w_{1..7}=$ `El agua moja y el fuego quema` $=[El, agua, moja, y, el, fuego, quema]$.

La siguiente matriz de embeddings, donde la i-ésima fila corresponde al vector de embedding de la i-ésima palabra, ordenadas según aparecen en la frase. (vectores de largo 2).
\begin{equation}
E = \begin{pmatrix}
2 & 2\\
0 & -2\\
0 & 1\\
-2 & 1\\
1 & 0\\
-1 & 1\\
1 & 1
\end{pmatrix}
\end{equation}

Los siguientes 3 filtros
\begin{equation}
U = \begin{pmatrix}
-1 & 1 & 0\\
1 & 1 & 0\\
0 & 0 & -1\\
1 & -1 & -1\\
-1 & -1 & 1\\
1 & 0 & -1
\end{pmatrix}
\end{equation}

Y la función de activación
\begin{equation}
tanh = \frac{e^{2x} - 1}{e^{2x} + 1}
\end{equation}

Usando estos paramátros escriba los pasos para calcular la representación (vector) resultante de aplicar la operación de convolución (sin padding) + max pooling. ¿De qué tamaño sería la ventana que debemos usar?

>Para que la convolución sin padding tenga sentido y se obtenga un valor escalar, se requiere que cada vector de características tenga la misma dimensión de los filtros, en este caso, 6. Para ello la ventana debe ser de tamaño 3. Con un stride de tamaño 1, se obtiene una matriz de características de dimensiones 5x6.
>
>$$ F = \begin{pmatrix}
>2 & 2 & 0 & -2 & 0 & 1\\
>0 & -2 & 0 & 1 & -2 & 1\\
>0 & 1 & -2 & 1 & 1 & 0\\
>-2 & 1 & 1 & 0 & -1 & 1\\
>1 & 0 & -1 & 1 & 1 & 1
>\end{pmatrix} $$
>
>Luego, al realizar la convolución con $U$ se obtiene una matriz de dimensiones 5x3, donde cada celda adquiere el siguiente valor:
>
>$$ \begin{array}{rcl}
>    C_{i,j} & = & \sum_{k=1}^6 F_{i,k}\cdot U_{6+1-k,j} \\
>    C & = & \begin{pmatrix}
>        -1 & -1 & 2 \\
>        -1 & 1 & -3 \\
>        -2 & 2 & 2 \\
>        -4 & -2 & 2 \\
>        0 & 3 & -1
>    \end{pmatrix}
>\end{array} $$
>
>A continuación, se aplica la función de activación sobre cada uno de los elementos de $C$. Dado que un vector $b$ no ha sido entregado se asume que es un vector de 0.
>
>$$ \tanh(C) = \begin{pmatrix}
>    -0.761 & -0.761 & 0.964 \\
>    -0.761 & 0.761 & -0.995 \\
>    -0.964 & 0.964 & 0.964 \\
>    -0.999 & -0.964 & 0.964 \\
>    0 & 0.995 & -0.761
>\end{pmatrix} $$
>
>Y finalmente, se filtran los valores por medio de un max pooling, al retornar el valor más grande por cada columna.
>
>$$ O = \begin{pmatrix} 0 & 0.995 & 0.964 \end{pmatrix} $$

# Recurrent Neural Networks (1 punto)


### Pregunta 4 (0,5 puntos)
Usando los embeddings de dos dimensiones de la pregunta anteror, la oración `el fuego quema` la podemos representar por una secuencia de vectores $(\vec{x}_1,\vec{x}_2,\vec{x}_3)$, con $\vec{x}_i \in \mathbb{R}^{d_x}$ y $d_x=2$.

Tenemos una red recurrente *Elman* definidad como:
\begin{equation}
\begin{split}
\vec{s}_i &= R_{SRNN}\left (\vec{x}_i, \vec{s}_{i-1}\right ) = g \left (\vec{s}_{i-1}W^s + \vec{x}_i W^x + \vec{b}\right ) \\
\vec{y}_i &= O_{SRNN}\left(\vec{s}_i\right) = \vec{s}_i \\
\end{split}
\end{equation}
donde
\begin{equation}
\vec{s}_i, \vec{y}_i \in \mathbb{R}^{d_s}, \quad W^x \in \mathbb{R}^{d_x \times d_s}, \quad W^s \in \mathbb{R}^{d_s \times d_s}, \quad \vec{b} \in \mathbb{R}^{d_s},
\end{equation}
y los vectores de estado $s_i$ son de tres dimensiones, $ds= 3$.

Sea
\begin{equation}
\begin{split}
\vec{s}_0 &= [0,0,0]\\
W^x &= \begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix} \\
W^s &= \begin{pmatrix}
1 & 0 &  1\\
0 & 1 & -1\\
1 & 1 &  1
\end{pmatrix} \\
\vec{b} &= [0, 0, 0] \\
g(x) &= ReLu(x) = max(0, x)
\end{split}
\end{equation}

<br>

Calcule manualmente los valores de los vectores $\vec{s}_1, \vec{s}_2,\vec{s}_3$ y de $\vec{y}_1, \vec{y}_2,\vec{y}_3$.

>Dado que $y_i = s_i$ sólo se requiere calcular los estados $s_i$ para obtener también las salidas de la RNN.
>
>Cálculo $s_1$
>$$ \begin{array}{rcl}
>s_1 & = & ReLu(s_0W^s + x_1W^x + b) \\
>s_1 & = & ReLu(\begin{pmatrix} 0 & 0 & 0 \end{pmatrix} \begin{pmatrix}
>1 & 0 &  1\\
>0 & 1 & -1\\
>1 & 1 &  1
>\end{pmatrix} + \begin{pmatrix} 1 & 0 \end{pmatrix} \begin{pmatrix}
>0 &  0 & 1\\
>1 & -1 & 0
>\end{pmatrix} + \begin{pmatrix} 0 & 0 & 0 \end{pmatrix}) \\
>s_1 & = & ReLu(\begin{pmatrix} 1 & 0 \end{pmatrix} \begin{pmatrix}
>0 &  0 & 1\\
>1 & -1 & 0
>\end{pmatrix}) \\
>s_1 & = & ReLu(\begin{pmatrix} 0 & 0 & 1 \end{pmatrix}) \\
>s_1 & = & \begin{pmatrix} 0 & 0 & 1 \end{pmatrix} \\
>y_1 & = & \begin{pmatrix} 0 & 0 & 1 \end{pmatrix}
>\end{array} $$
>
>
>Cálculo $s_2$
>$$ \begin{array}{rcl}
>s_2 & = & ReLu(s_1W^s + x_2W^x + b) \\
>s_2 & = & ReLu(\begin{pmatrix} 0 & 0 & 1 \end{pmatrix} \begin{pmatrix}
>1 & 0 &  1\\
>0 & 1 & -1\\
>1 & 1 &  1
>\end{pmatrix} + \begin{pmatrix} -1 & 1 \end{pmatrix} \begin{pmatrix}
>0 &  0 & 1\\
>1 & -1 & 0
>\end{pmatrix} + \begin{pmatrix} 0 & 0 & 0 \end{pmatrix}) \\
>s_2 & = & ReLu(\begin{pmatrix} 1 & 1 & 1 \end{pmatrix} + \begin{pmatrix} 1 & -1 & -1 \end{pmatrix}) \\
>s_2 & = & ReLu(\begin{pmatrix} 2 & 0 & 0 \end{pmatrix}) \\
>s_2 & = & \begin{pmatrix} 2 & 0 & 0 \end{pmatrix} \\
>y_2 & = & \begin{pmatrix} 2 & 0 & 0 \end{pmatrix}
>\end{array} $$
>
>
>Cálculo $s_3$
>$$ \begin{array}{rcl}
>s_3 & = & ReLu(s_2W^s + x_3W^x + b) \\
>s_3 & = & ReLu(\begin{pmatrix} 2 & 0 & 0 \end{pmatrix} \begin{pmatrix}
>1 & 0 &  1\\
>0 & 1 & -1\\
>1 & 1 &  1
>\end{pmatrix} + \begin{pmatrix} 1 & 1 \end{pmatrix} \begin{pmatrix}
>0 &  0 & 1\\
>1 & -1 & 0
>\end{pmatrix} + \begin{pmatrix} 0 & 0 & 0 \end{pmatrix}) \\
>s_3 & = & ReLu(\begin{pmatrix} 2 & 0 & 2 \end{pmatrix} + \begin{pmatrix} 1 & -1 & 1 \end{pmatrix}) \\
>s_3 & = & ReLu(\begin{pmatrix} 3 & -1 & 3 \end{pmatrix}) \\
>s_3 & = & \begin{pmatrix} 3 & 0 & 3 \end{pmatrix} \\
>y_3 & = & \begin{pmatrix} 3 & 0 & 3 \end{pmatrix}
>\end{array} $$

### Pregunta 5 (0.5 puntos)
¿De qué forma las RNN y las CNN logran aprender representaciones específicas
para la tarea objetivo? Compare la forma en que las RNN y las CNN aprenden con los modelos que usan *features* diseñadas manualmente.

**Respuesta**

Tanto las CNN como las RNN son redes neuronales que aprenden gracias a la minimización de una función de pérdidas, utilizando como heurística alguna variante del algoritmo de gradiente descendente. Cabe destacar que este tipo de configuraciones permite obtener información sobre todo el texto entregado, en redes CNN por medio de las operaciones de pooling y en redes RNN por medio de la recurrencia de la red. Por otra parte, los modelos manuales son generalemente modelos markovianos que asumen independencia entre tokens, de modo que no logran capturar toda la información contenida en el texto.

# Pregunta 6: Redes Neuronales con Pytorch (3 puntos) 💬

<center>
<img src="https://www.anda.cl/wp-content/uploads/2021/03/0_5vNAtimPjYQr4W72.gif" alt="chatbot" width="400">
</center>

En esta sección de la tarea deberán implementar un Chatbot que sea capaz de generar una conversación *“básica”* utilizando un dataset de *Star Wars*. **El objetivo** de esta pregunta es que puedan aplicar lo aprendido sobre redes neuronales utilizando Pytorch en un ejemplo práctico.  Durante el desarrollo, se espera que puedan diseñar un bot (que tendrá por atrás un clasificador) que sea capaz de clasificar diferentes etiquetas, cosa que una vez identificada la etiqueta entregue una respuesta acorde a lo preguntado.

**Aviso:** Antes de comenzar con una descripción mas profunda de esta sección, les recomendamos que visualicen y se familiaricen con el dataset entregado, de esta forma comprenderán mejor la descripción del enunciado (aquí una pequeña ayudita 🆘).

In [91]:
import pandas as pd

star_wars_chatbot_path = "../../data/star_wars_chatbot.json"
example_data = pd.read_json(star_wars_chatbot_path)
print("Cantidad de tags: ", example_data['intents'].shape[0])

Cantidad de tags:  16


In [92]:
print( "Keys de cada fila:", example_data.head(10).iloc[0]['intents'].keys() )
print( "Ejemplo de tag:", example_data.head(10).iloc[0]['intents']['tag'] )
print( "Ejemplo de patterns:", example_data.head(10).iloc[0]['intents']['patterns'] )
print( "Ejemplo de responses:", example_data.head(10).iloc[0]['intents']['responses'] )

Keys de cada fila: dict_keys(['tag', 'patterns', 'responses'])
Ejemplo de tag: greeting
Ejemplo de patterns: ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day', "What's up", 'Yo!', 'Howdy', 'Nice to meet you.']
Ejemplo de responses: ['Hey', 'Hello, thanks for visiting.', 'Hi there, what can I do for you?', 'Hi there, how can I help?', 'Hello, there.', 'Hello Dear', 'Ooooo Hello, looking for someone or something?', 'Yes, I am here.', 'Listening carefully.', 'Ok, I am with you.']


A continuación, ejemplos del contenido del primer registro:

In [93]:
example_data['intents'][0]['patterns']

['Hi',
 'Hey',
 'How are you',
 'Is anyone there?',
 'Hello',
 'Good day',
 "What's up",
 'Yo!',
 'Howdy',
 'Nice to meet you.']

In [94]:
example_data['intents'][0]['responses']

['Hey',
 'Hello, thanks for visiting.',
 'Hi there, what can I do for you?',
 'Hi there, how can I help?',
 'Hello, there.',
 'Hello Dear',
 'Ooooo Hello, looking for someone or something?',
 'Yes, I am here.',
 'Listening carefully.',
 'Ok, I am with you.']

In [95]:
example_data['intents'][0]['tag']

'greeting'

Del dataset cargado podemos notar que este viene en un formato `JSON`, por lo que sus datos están almacenados en diccionarios. Las llaves de los diccionarios no son aleatorias y estos nos sirven para identificar puntos relevantes en el desarrollo del bot. A continuación, se realiza una pequeña descripción de las llaves:

- `patterns`: Almacena los patrones con los que entrenaremos el modelo 😮, en otras palabras, es el corpus de entrenamiento que contiene solo preguntas o expresiones que deberá responder el bot.
- `responses`: Son las respuestas 🙋 relacionadas a los `patterns`, estas las utilizaremos en una etapa posterior a la clasificación, para dar una respuesta aleatoría al usuario.
- `tag`: Son las labels con las que entrenaremos nuestro modelo 💻.

En síntesis, las `keys` relevantes para el entrenamiento de nuestra red neuronal serán `patterns` (corpus) y `tag` (etiquetas).

#### Explicación de la tarea a realizar:

**Explicación de la tarea a realizar**: Implemente una Class llamada `CNNClassifier` que sea capaz de entrenar un modelo de texto a través de una red neuronal Feed Forward y una arquitectura convolucional (CNN 1D) [`torch.nn.Conv1d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html#conv1d) . Para el diseño de las redes tienen completa libertad, pero se le aconseja que se guíen de la última auxiliar para la construcción. Es **importantísimo** que el modelo a crear posea una capa de `Embedding` que se genere en base al entrenamiento del modelo. Creado el modelo, construya una función batch para cargar los datos de entrenamiento del modelo.

Construido el modelo, compare los resultados obtenidos para una red feed forward y una cnn. Para la comprobación de sus resultados ejecute el chatbot y pruebelo, ¿qué configuración tiene mejores resultados?, ¿a qué se deberan estos resultados?

Ojo que un ejemplo de prueba con el chatbot puede ser (agregue mas preguntas ud):

```
Let's chat! (type 'finish_chat' to finish the chat)
You: hi
GA-97: Yes, I am here.
You: can you tell me a joke?
GA-97: Have you tried the gluten-free Wookiee treats? No, but I heard they are a little Chewy.
```

El resto del código referido a la ejecución del chat se los entregamos, por lo que no deberían tener mayores problemas 😸 (en caso de tener problemas con su código, puede modificar cualquier parte sugerida siempre y cuando cumpla lo solicitado).

**Igual [mucho texto](https://i0.wp.com/elgeneracionalpost.com/wp-content/uploads/2020/07/mucho-texto.jpg?fit=1280%2C720&ssl=1).... En resumen, ¿Qué se solicita?:**

- [ ] Diseñar una red neuronal Feed Forward.
- [ ] Diseñar un red convolucional.
- [ ] Utilizar una capa de embeddings para generar representaciones vectoriales del corpus.
- [ ] Crear el método forward de la clase `CNNClassifier`.
- [ ] Crear la función BATCH.
- [ ] Probar el modelo y comparar los resultados obtenidos con la red Feed Forward y la red CNN. Comente sus resultados de forma cualitativa, señalando con qué tipo de red obtuvo mejores resultados con el chatbot.

**Nota-1:** El modelo creado debe tener la opción de entrenar a traves de una feed forward y una CNN. Esto no significa que entrenará una FF y una CN, el modelo deberá recibir un booleano que especifique que tipo de red utilizará.

**Nota-2:** El dataset se descargará automáticamente en la sección `Carga de Dataset 📚`, no os preocupéis.

#### Pasemos al Código 🦾

Esqueleto propuesto (se **RECOMIENDA** que cambien **SOLO** la red neuronal y la función Batch) 🦴:

##### Instalamos librerias necesarias e importamos 😀

In [96]:
import json
import random
from itertools import zip_longest

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader

from torch.optim import SGD, lr_scheduler

##### Carga de Dataset 📚

In [97]:
# Load the dataset using json
with open(star_wars_chatbot_path, 'r') as f:
    dataset = json.load(f)

# Create a vocab with the dataset and get the number of classes that have
tokenizer = get_tokenizer("basic_english")
vocab = build_vocab_from_iterator(tokenizer(x) for list_words in dataset['intents'] for x in list_words['patterns'])
num_classes = len(dataset['intents'])

# Define a list with the labels
labels = sorted(set([tag for tag in [intents['tag'] for intents in dataset['intents']]]))
# Define a train_list where we can find the info in the format: [(tag_0, text_0)...,(tag_n-1, text_n-1)]
train_list = [(labels.index(intents['tag']), text) for intents in dataset['intents'] for text in intents['patterns']]

Se añade un token para tokens desconocidos

In [98]:
UNK_IDX = 0
vocab.insert_token('<pad>', 0)
vocab.set_default_index(UNK_IDX)

stoi = vocab.get_stoi()

##### Creación del modelo (2 puntos en total)

In [99]:
# Construya el modelo
class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, num_classes=10,
                 use_cnn=False, cnn_pool_channels=24, cnn_kernel_size=3):
        super().__init__()

        self.cnn_kernel_size = cnn_kernel_size
        self.use_cnn = use_cnn

        # Capa de embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        if use_cnn:
            # Capa de convolución
            self.conv = nn.Conv1d(
                in_channels = 1, # un canal de entrada
                out_channels = cnn_pool_channels,
                kernel_size = cnn_kernel_size * embed_dim,
                stride = embed_dim
            )

            # Tamaño de entrada de capa lineal
            fc_in_size = cnn_pool_channels

            # Creación capa lineal
            self.fc = nn.Linear(fc_in_size, num_classes)

            # Inicializacion de pesos de las capas
            self.init_weights()

        else:
            # capas de la MLP
            self.fc = nn.Linear(embed_dim, num_classes)


    def init_weights(self):
        # Valor inicial de los pesos
        initrange = 0.5

        # Inicialización de pesos en capa de embedding
        self.embedding.weight.data.uniform_(-initrange, initrange)

        # Inicialización de pesos en la capa lineal
        self.fc.weight.data.uniform_(-initrange, initrange)

        # Inicilización de bias de la capa lineal en cero
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        # Preparamos el input de la capa de embeddings a partir de text y offsets
        text = torch.tensor(
            list(
                zip(
                    *zip_longest(
                        *([text[o:offsets[i+1]] for i, o in enumerate(offsets[:-1])] + [text[offsets[-1]:len(text)]]),
                        fillvalue=vocab["<pad>"]
                    )
                )
            )
        ).to(text.device)
        if text.shape[1] < self.cnn_kernel_size:
            zeros = torch.zeros((text.shape[0], self.cnn_kernel_size - text.shape[1]), dtype=torch.long).to(text.device)
            text = torch.cat((text, zeros), dim=1).to(text.device)

        # Obtenemos la representación de la frase a partir de la capa de embedding
        h = self.embedding(text)

        if self.use_cnn:
            # Aplicamos la capa de convolución
            h = h.view(h.size(0), 1, -1)
            h = torch.relu(self.conv(h))
            h = h.mean(dim=2)

            # Obtenemos el resultado final a partir de la capa lineal
            output = self.fc(h)

        else:
            # La representacion de un documento sera el promedio de los
            # embeddings de sus palabras.
            h = h.mean(dim=1)

            # computar las capas de la red MLP
            output = self.fc(h)
        
        # Aplicamos la función de activación log-softmax
        return F.log_softmax(output, dim=1)

##### Función Batch 👷 (0,5 puntos)

In [100]:
# Defina su función de BATCH
def generate_batch(batch):#, use_cnn):
    # if use_cnn:
    label = torch.tensor([ entry[0] for entry in batch ])
    texts = [ tokenizer(entry[1]) for entry in batch ]
    offsets = [0] + [len(text) for text in texts]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    big_text = torch.cat([torch.tensor([vocab[t] if t in stoi else 0 for t in text]) for text in texts])

    return big_text, offsets, label

##### Entrenamiento 🥊

In [101]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"GPU is avaible: {device}")

# Define the different inputs in our model
num_epochs = 1000
BATCH_SIZE = 16
LR = 1e-1
INPUT_SIZE = len(vocab)
OUTPUT_SIZE = num_classes
USE_CNN = True
SHUFFLE = True

# Define model, optimizer, loss and scheduler (Q: ¿What is it?)
model = CNNClassifier(INPUT_SIZE, num_classes=OUTPUT_SIZE, use_cnn=USE_CNN).to(device)
optimizer = SGD(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss().to(device)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: .9 ** (epoch // 10)])

print(f'train: {len(train_list)} elements')

# We train the model using the intents
loss_list= []
for epoch in range(1, num_epochs):
  train_loader = DataLoader(train_list, shuffle=SHUFFLE, batch_size=BATCH_SIZE, collate_fn=generate_batch)
  model.train()
  total_loss = 0
  for i, (texts, offsets, cls) in enumerate(train_loader):
    texts = texts.to(device)
    offsets = offsets.to(device)
    cls = cls.to(device)
    optimizer.zero_grad()
    output = model(texts, offsets)
    loss = criterion(output, cls)
    total_loss += loss.item()
    loss.backward()
    optimizer.step()

  loss_list.append(loss.item())
  print('\rEpoch: {0:03d} \t iter-Loss: {1:.3f}'.format(epoch+1, loss.item()))

print(f'final loss: {loss.item():.4f}')

GPU is avaible: cuda
train: 97 elements
Epoch: 002 	 iter-Loss: 2.799
Epoch: 003 	 iter-Loss: 3.125
Epoch: 004 	 iter-Loss: 2.853
Epoch: 005 	 iter-Loss: 2.792
Epoch: 006 	 iter-Loss: 2.637
Epoch: 007 	 iter-Loss: 2.396
Epoch: 008 	 iter-Loss: 2.904
Epoch: 009 	 iter-Loss: 2.601
Epoch: 010 	 iter-Loss: 2.941
Epoch: 011 	 iter-Loss: 2.331
Epoch: 012 	 iter-Loss: 2.999
Epoch: 013 	 iter-Loss: 2.078
Epoch: 014 	 iter-Loss: 2.156
Epoch: 015 	 iter-Loss: 3.103
Epoch: 016 	 iter-Loss: 2.518
Epoch: 017 	 iter-Loss: 2.053
Epoch: 018 	 iter-Loss: 3.181
Epoch: 019 	 iter-Loss: 1.550
Epoch: 020 	 iter-Loss: 3.078
Epoch: 021 	 iter-Loss: 2.376
Epoch: 022 	 iter-Loss: 1.802
Epoch: 023 	 iter-Loss: 3.094
Epoch: 024 	 iter-Loss: 2.913
Epoch: 025 	 iter-Loss: 1.412
Epoch: 026 	 iter-Loss: 1.995
Epoch: 027 	 iter-Loss: 1.876
Epoch: 028 	 iter-Loss: 2.909
Epoch: 029 	 iter-Loss: 2.989
Epoch: 030 	 iter-Loss: 3.167
Epoch: 031 	 iter-Loss: 1.058
Epoch: 032 	 iter-Loss: 3.596
Epoch: 033 	 iter-Loss: 2.371


##### A probar! 🧪

In [102]:
# This is working?, Try the next example!
qText = "Do you know any joke?" # this must classify the label "funny"

# X = torch.tensor([vocab.stoi[t] for t in tokenizer(qText)]).to(device)
X = torch.tensor([vocab[t] if t in stoi else 0 for t in tokenizer(qText)]).to(device)
if X.shape[0] < model.cnn_kernel_size:
    pad = torch.zeros((model.cnn_kernel_size - X.shape[0]), dtype=torch.long).to(X.device)
    X = torch.concat([X, pad])

model.eval()
output = model(X, torch.tensor([0], dtype=torch.long).to(device))
_, predicted = torch.max(output, dim=1)
labels[predicted]

'funny'

Ya pero prometiste hacer un chatbot, no una simple clasificación....

##### Guardamos modelo 🦺 (opcional)

In [103]:
# We save de model using pytorch (this is optional, just to learn how to do this in pytorch)
data = {
    "model_state": model.state_dict(),
    "input_size": INPUT_SIZE,
    "output_size": OUTPUT_SIZE,
    "use_cnn": USE_CNN,
    "labels": labels
}

FILE_PATH = "../../models/tarea4.pth"
torch.save(data, FILE_PATH)

print(f'training complete. file saved to {FILE_PATH}')

training complete. file saved to ../../models/tarea4.pth


##### Chatbot 💬

In [104]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open(star_wars_chatbot_path, 'r') as json_data:
    intents = json.load(json_data)

FILE_PATH = "../../models/tarea4.pth"
data = torch.load(FILE_PATH)

INPUT_SIZE = data["input_size"]
OUTPUT_SIZE = data["output_size"]
USE_CNN = data["use_cnn"]
labels = data['labels']
model_state = data["model_state"]

model = CNNClassifier(INPUT_SIZE, num_classes=OUTPUT_SIZE, use_cnn=USE_CNN).to(device)
model.load_state_dict(model_state)
model.eval()

# Dictionary with the answers
responses = {key['tag']: key['responses'] for key in dataset['intents']}

bot_name = "GA-97"
print("Let's chat! (type 'finish_chat' to finish the chat)")
while True:
    q_text = input("You: ")
    if q_text == 'finish_chat':
        break

    # X = torch.tensor([vocab.stoi[t] for t in tokenizer(q_text)]).to(device)
    X = torch.tensor([vocab[t] if t in stoi else 0 for t in tokenizer(q_text)]).to(device)
    if X.shape[0] < model.cnn_kernel_size:
        pad = torch.zeros((model.cnn_kernel_size - X.shape[0]), dtype=torch.long).to(X.device)
        X = torch.concat([X, pad])
    output = model(X, torch.tensor([0], dtype=torch.long).to(device))
    _, predicted = torch.max(output, dim=1)

    tag = labels[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.50:
      print(f"{bot_name}: {random.choice(responses[tag])}")
    else:
      print(f"{bot_name}: My model can't understand you...")

Let's chat! (type 'finish_chat' to finish the chat)


GA-97: Hello, thanks for visiting.
GA-97: Hey
GA-97: I can talk and do things for you
GA-97: I can talk and do things for you
GA-97: I'm in doubt about that
GA-97: Menu: Fuzzy Tauntaun, Bloody Rancor, Jedi Mind Trick, T-16 Skyhopper, Yub Nub, Jet Juice, Hyperdrive, Rancor Beer.
GA-97: Whas is you problem?
GA-97: Ask for jedi or sith or bounti hounter.
GA-97: Luke Skywalker, Yoda, Obi-Wan Kenobi, Anakin Skywalker, Qui-Gon Jinn, Mace Windu, Ahsoka Tano, Plo Koon, Aalya Secura, Kit Fisto.
GA-97: Here is top 10 sith you are looking for: Darth Vader, Darth Plagueis, Darth Revan, Darth Traya, Darth Sidious, Darth Maul, Ulic Qel-Droma, Asajj Ventress, Kylo Ren, Marka Ragnos.
GA-97: Here is top 10 bounti hounter you are looking for: Jango Fett, Boba Fett, Cad Bane, Durge, Embo, Dengar, Black Krrsantan, IG-88, Aurra Sing, Sabine Wren.
GA-97: You are a dumb person asking a machine about yourself
GA-97: Mr. Fbravo is my creator.
GA-97: A wise and intelligent man
GA-97: You would get bored if I do

#### Comente los resultados aquí (0,5 puntos)

>El modelo es capaz de responder correctamente a todos los prompts con los que se ha entrenado, pues al ser un ejemplo introductorio no se ha verificado la existencia de overfitting, ni se ha separado la data de entrenamiento en train/val/test. En general suele entregar respuestas acorde a lo que se pide, excepto cuando existen 2 o más tokens muy relacionado a alguna categoria. Por ejemplo, al usar la palabra "Tell" junto a "joke" es muy posible que el modelo se incline por determinar que la categoría es "story", "myself" u otra en vez de "funny", entregando una respuesta distinta a la esperada.